# [Solar X-ray Monitor (XSM) on-board Chandrayaan 2](https://www.prl.res.in/ch2xsm/xsm_instrument)

Solar X-ray Monitor (abbreviated as XSM) observes the Sun as a star and measures the solar X-ray spectrum in the energy range of 1-15 keV with an energy resolution of ~175 eV at 5.9 keV and a time cadence of one second. The broadband soft X-ray spectra with XSM provides diagnostics of the solar corona at various levels of solar activity from quiescent Sun to large solar flares. 

## Specification

|                       |  |
| --------------------------------- | -------------------------------------------------------------- |
| Energy Range                      | 1 – 15 Kev (up to ≈ M5 class)<br>2 – 15 keV (above ≈ M5 class) |
| Energy Resolution                 | ~175 eV @ 5.9 Kev                                              |
| Time Cadence                      | 1 second                                                       |
| Effective area (on-axis)          | 0.135 mm<sup>2</sup> @ 1 keV<br>0.367 mm<sup>2</sup> @ 5 keV   |
| Field of View                     | ± 40 degree                                                    |
| Filter wheel positions            | 3: Open, Be-filter, Cal(Fe-55)                                 |
| Be-filter movement threshold flux | 80,000 counts s<sup>−1</sup> (≈ M5 class)                      |

## Science Objectives

**Flare Plasma Diagnostics:**

The intensity of soft X-ray radiation (SXR) in the 1–15 keV range is determined by the temperature (T) and the emission measure (EM) of the heated flare plasma under the assumption of a bremsstrahlung mechanism.

## Sherpa (Fitting Application)

Sherpa is a modeling and fitting application for Python. It contains a powerful language for combining simple models into complex expressions that can be fit to the data using a variety of statistics and optimization methods. It is easily extensible to include user models, statistics, and optimization methods. It provides a high-level User Interface for interactive data-analysis work, such as within a Jupyter notebook, and it can also be used as a library component, providing fitting and modeling capabilities to an application.

## Sunxspex (Modeling Application)
Sunxspex computes thermal and non-thermal solar x-ray emission.

In [ ]:
from sunxspex.thermal import thermal_emission
from sunxspex import emission
import astropy.units as u
import numpy as np

from sherpa.astro.instrument import RSPModelPHA
from sherpa.astro.instrument import RSPModelNoPHA
from sherpa.astro.io import read_arf, read_rmf, read_pha
from sherpa.models import model
from sherpa.astro.instrument import Response1D
from sherpa.stats import Chi2, CStat, Cash

from sherpa.fit import Fit
from astropy.io import fits
import datetime

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use(['science','notebook','grid'])

# XSM Data
## Lightcurve Data

In [ ]:
lc_file = 'xsm_data/20220328/calibrated/ch2_xsm_20220328_v1_level2.lc'
pha_file = 'xsm_data/20220328/calibrated/ch2_xsm_20220328_v1_level2.pha'

# hdus = fits.open('xsm_data/20210908/calibrated/ch2_xsm_20210908_v1_level2.lc')
hdus = fits.open(lc_file)
lc_data = hdus[1].data
tm = lc_data['TIME'] - np.min(lc_data['TIME'])
rate = lc_data['RATE']
rate_err = lc_data['ERROR']

fig, ax = plt.subplots()
# fig.set_size_inches(16,9)
ax.errorbar(tm,rate,yerr=rate_err,lw=3)
ax.set_yscale('log')
# ax.grid()
ax.set_xlabel('Time (s)')
ax.set_ylabel('XSM counts/s')
ax.set_title('XSM Lightcurve Data');

## Spectral Data

In [ ]:
all_phas = read_pha(pha_file,use_errors=True);
print(f'Number of spectra found: {len(all_phas)}')

In [ ]:
pha_tmp1 = all_phas[200]
pha_tmp2 = all_phas[700]

In [ ]:
from sherpa.plot import *
from sherpa.astro.plot import DataPHAPlot

In [ ]:
dplot1 = DataPHAPlot()
dplot1.histo_prefs['linestyle'] = '-'
dplot1.prepare(pha_tmp1)
dplot1.plot(ylog=True,linestyle='solid')
dplot1.prepare(pha_tmp2)
dplot1.overplot(ylog=True,linestyle='solid')

## Instrument Response

### Response Matrix Function

In [ ]:
rmf = read_rmf('caldb/CH2xsmrspwitharea_open20191214v01.rsp')
rmf

### Effective Area

In [ ]:
effarea=np.loadtxt('caldb/xsm_onaxis_effective_area.txt')

fig, ax = plt.subplots()
ax.plot(rmf.e_max,effarea)
ax.set_xlabel('Energy (keV)')
ax.set_ylabel(r'Effective Area (cm$^2$)');

In [ ]:
pha_tmp1.set_rmf(rmf)
pha_tmp1.set_analysis('energy')

In [ ]:
dplot1 = DataPHAPlot()
dplot1.histo_prefs['linestyle'] = '-'
dplot1.prepare(pha_tmp1)
dplot1.plot(ylog=True,linestyle='solid')

# XSM Data Analysis

## Data Seletion

In [ ]:
effarea=np.loadtxt('caldb/xsm_onaxis_effective_area.txt')



def xsm_gen_lc(spec_file,ene_lo,ene_hi):
    hdu1 = fits.open(spec_file)
    hdu=fits.BinTableHDU.from_columns(hdu1[1].columns)

    data=hdu.data

    time_xsm = (data['TSTART']+data['TSTOP'])/2.
    tbinsize=(data['TSTOP'][0]-data['TSTART'][0])
    
    exposure=data['EXPOSURE']
    
    ns=len(data)
    fluxlc = np.zeros(ns)
    countlc = np.zeros(ns)
    
    ene=np.arange(0,512)*0.033+0.5+0.033/2.0 # Midpoint energy of PI bins
    
    kev2erg=1.6021e-9
    cgs2si=0.001

    startch=int((ene_lo-0.5)/0.033)
    stopch=int((ene_hi-0.5)/0.033)


    for i in range(0,ns):
        spec = data['COUNTS'][i]/data['EXPOSURE'][i]
        spec_ene = spec*ene/effarea
        fluxlc[i] = np.sum(spec_ene[startch:stopch])*kev2erg*cgs2si
        countlc[i] = np.sum(spec[startch:stopch])
    
    ## Select only bins with atleast half exposure for tbinsize >=10
    if (tbinsize >= 10.0):
        ind=(exposure  > max(exposure)/2.0)
        time_xsm=time_xsm[ind]
        fluxlc=fluxlc[ind]
        countlc = countlc[ind]
        exposure=exposure[ind]


    nbins=int(86400.0/tbinsize)
    
    tday0=int(time_xsm[0]/86400.0)*86400.0
    t0=(time_xsm[0]-int((time_xsm[0]-tday0)/tbinsize)*tbinsize)
    alltime=np.arange(0,nbins)*tbinsize+t0
    allflux=np.empty(nbins)
    allflux[:]=np.nan
    allcount=np.empty(nbins)
    allcount[:]=np.nan

    for i,t in enumerate(time_xsm):
        tbin=int((t-tday0)/tbinsize)
        allflux[tbin]=fluxlc[i]
        allcount[tbin]=countlc[i]

    return alltime, allflux, allcount


ene_lo = 1.551 #keV
ene_hi = 12.408 #keV

# alltime, allflux, allcount = xsm_gen_lc('xsm_data/20210908/calibrated/ch2_xsm_20210908_v1_level2.pha',ene_lo,ene_hi)
alltime, allflux, allcount = xsm_gen_lc(pha_file,ene_lo,ene_hi)
alltime = alltime - alltime.min()

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(alltime,allflux,label=r'1-8 $\AA$')
ax.set_xlabel('Time (s)')
ax.set_ylabel(r'Flux (W/m$^2$)')
ax.set_title('XSM Lightcurve Data (Rebinned)');
ax.legend();

### [Solar Flare M4.0](https://www.solarmonitor.org/goes_pop.php?date=20220328&type=xray)

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(alltime[650:800],allflux[650:800])
ax.set_xlabel('Time (s)')
ax.set_ylabel(r'Flux (W/m$^2$)')
ax.set_title('M4.0 Flare');

In [ ]:
alltime, allflux_1_3, allcount = xsm_gen_lc(pha_file,1,3)
alltime, allflux_3_6, allcount = xsm_gen_lc(pha_file,3,6)
alltime, allflux_6_10, allcount = xsm_gen_lc(pha_file,6,10)
alltime, allflux_10_15, allcount = xsm_gen_lc(pha_file,10,15)
alltime = alltime - alltime.min()

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(alltime[600:900],allflux_1_3[600:900],label=r'$1 - 3$ keV')
ax.semilogy(alltime[600:900],allflux_3_6[600:900],label=r'$3 - 6$ keV')
ax.semilogy(alltime[600:900],allflux_6_10[600:900],label=r'$6 - 10$ keV')
ax.semilogy(alltime[600:900],allflux_10_15[600:900],label=r'$10 - 15$ keV')
ax.set_xlabel('Time (s)')
ax.set_ylabel(r'Flux (W/m$^2$)')
ax.legend(loc=4)
ax.set_title('M4.0 Flare');

In [ ]:
alltime, allflux_1_3, allcount = xsm_gen_lc(pha_file,1,3)
alltime, allflux_3_6, allcount = xsm_gen_lc(pha_file,3,6)
alltime, allflux_6_10, allcount = xsm_gen_lc(pha_file,6,10)
alltime, allflux_10_15, allcount = xsm_gen_lc(pha_file,10,15)
# alltime = alltime - alltime.min()
fig, ax = plt.subplots()
ax.semilogy(alltime[600:900],allflux_1_3[600:900],label=r'$1 - 3$ keV')
ax.semilogy(alltime[600:900],allflux_3_6[600:900],label=r'$3 - 6$ keV')
ax.semilogy(alltime[600:900],allflux_6_10[600:900],label=r'$6 - 10$ keV')
ax.semilogy(alltime[600:900],allflux_10_15[600:900],label=r'$10 - 15$ keV')
ax.axvspan(alltime[683],alltime[687],alpha=0.5, color='black')
ax.set_xlabel('Time (s)')
ax.set_ylabel(r'Flux (W/m$^2$)')
ax.legend()
ax.xaxis.set_tick_params(labelsize=12)
ax.set_title('M4.0 Flare (UT)');

In [ ]:
def xsm_genspec(spec_file,tstart,tstop,outfile): # times in seconds since 2017,1,1
    tref = datetime.datetime(2017, 1, 1, 0, 0)
    hdu1 = fits.open(spec_file)
    hdu=fits.BinTableHDU.from_columns(hdu1[1].columns)

    data=hdu.data

    time_xsm = (data['TSTART']+data['TSTOP'])/2.
    tbinsize=(data['TSTOP'][0]-data['TSTART'][0])
    
    exposure=data['EXPOSURE']

    inds = (time_xsm >= tstart) & (time_xsm <= tstop)

    data_f = data[inds]

    channel = data_f[0][1]
    spec_data = np.zeros(512)
    stat_err = np.zeros(512)
    sys_err = np.zeros(512)
    exposure = 0

    for di in data_f:
        spec_data = spec_data + di[2]
        stat_err = stat_err + di[3]
        sys_err = sys_err + di[4]
        exposure = exposure + di[5]

    # writing file
    hdu_list = []
    primary_hdu = fits.PrimaryHDU()
                                    
    hdu_list.append(primary_hdu)

    fits_columns = []
    col1 = fits.Column(name='CHANNEL',format='J',array=channel)
    col2 = fits.Column(name='COUNTS',format='E',array=spec_data)
    col3 = fits.Column(name='STAT_ERR',format='E',array=stat_err)
    col4 = fits.Column(name='SYS_ERR',format='E',array=sys_err)

    fits_columns.append(col1)
    fits_columns.append(col2)
    fits_columns.append(col3)
    fits_columns.append(col4)

    hdu_pha = fits.BinTableHDU.from_columns(fits.ColDefs(fits_columns))
    hdu_pha.name = 'SPECTRUM'
                                                                       
    hdu_list.append(hdu_pha)
                                                                       
    _hdu_list = fits.HDUList(hdus=hdu_list)

    tstart_dt = datetime.datetime.fromtimestamp(tstart+tref.timestamp())
    tstop_dt = datetime.datetime.fromtimestamp(tstop+tref.timestamp())

    _hdu_list[1].header.set('TSTART',tstart_dt.isoformat())
    _hdu_list[1].header.set('TSTOP',tstop_dt.isoformat())
    _hdu_list[1].header.set('EXPOSURE',f'{exposure:.2f}')
    _hdu_list.writeto(f'{outfile}.pha')


In [ ]:
xsm_genspec(pha_file,alltime[683],alltime[687],'ch2_xsm_20220328_v1_level2_peak')

## Fitting Spectral Data

### Loading Spectral Data

In [ ]:
pha = read_pha('ch2_xsm_20220328_v1_level2_peak.pha',use_errors=True)
pha.set_rmf(rmf)
pha.set_analysis('energy')
pha.notice(1.5,15)

pha.staterror[pha.staterror==0] = 1
pha.syserror[pha.syserror == 0] = 1
print(pha)

In [ ]:
dplot1 = DataPHAPlot()
dplot1.prepare(pha)
dplot1.histo_prefs['linestyle'] = '-'
dplot1.plot(ylog=True,linestyle='solid')

### Model (Thermal Emission)

In [ ]:
class f_vth_sherpa(model.ArithmeticModel):
    def __init__(self,name='f_vth'):
        self.temperature = model.Parameter(name,'Temperature',10,min=1,hard_min=0)
        self.em = model.Parameter(name,'EM',1,min=1e-5,hard_min=1e-49)
        model.ArithmeticModel.__init__(self, name,(self.temperature,self.em))

    def calc(self,pars,x,*args,**kwargs):
        inds = (x>1.0002920302956426) & (x<100)
        x = x[inds]
        x = np.append(x,2*x[-1]-x[-2])
        ene = x*u.keV
        temp = pars[0]*1e6*u.K
        em = pars[1]*1e49/(u.cm**5)

        y = thermal_emission(ene,temp,em)

        spec = np.zeros(len(inds))
        spec[inds] = y

        return spec

mdl = f_vth_sherpa()
ene_x = pha.get_x()
print(mdl)

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(ene_x,mdl.calc([5,1],ene_x),label='5 MK')
ax.semilogy(ene_x,mdl.calc([10,1],ene_x),label='10 MK')
ax.semilogy(ene_x,mdl.calc([20,1],ene_x),label='20 MK')
ax.semilogy(ene_x,mdl.calc([30,1],ene_x),label='30 MK')
ax.legend()
ax.set_xlabel('Energy (keV)')
ax.set_ylabel('Flux')
ax.set_xlim([1,20]);

In [ ]:
xsm_rsp = Response1D(pha)
mdl_rsp = xsm_rsp(mdl)
print(mdl_rsp)

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(ene_x,mdl_rsp.calc([5,0.0018],ene_x),label='5 MK')
ax.semilogy(ene_x,mdl_rsp.calc([10,0.0018],ene_x),label='10 MK')
ax.semilogy(ene_x,mdl_rsp.calc([20,0.0018],ene_x),label='20 MK')
ax.semilogy(ene_x,mdl_rsp.calc([30,0.0018],ene_x),label='30 MK')
ax.semilogy(ene_x,pha.get_y(),label='Observation')
ax.set_xlabel('Energy (keV)')
ax.set_ylabel('XSM Counts')
ax.legend()
ax.set_xlim([1,15]);

### Fitting to the model

In [ ]:
pha = read_pha('ch2_xsm_20220328_v1_level2_peak.pha',use_errors=True)
pha.set_rmf(rmf)
pha.set_analysis('energy')
pha.notice(1.5,15)

pha.staterror[pha.staterror==0] = 1
pha.syserror[pha.syserror == 0] = 1

mdl_rsp.model.pars[0].val = 10 # Temperature
mdl_rsp.model.pars[1].val = 0.01 #EM

fi = Fit(pha,mdl_rsp)

fres = fi.fit()
print(fres)
print(f'Reduced Chi2 = {fres.statval/fres.dof:.4f}')

In [ ]:
dplot = DataPHAPlot()
dplot.histo_prefs['linestyle'] = '-'
dplot.prepare(fi.data)
mplot = ModelPlot()
mplot.prepare(fi.data, fi.model)
dplot.plot(ylog=True)
mplot.overplot(color='red')

In [ ]:
fplot = FitPlot()
rplot = DelchiPlot()
splot = SplitPlot()
dplot = DataPHAPlot()
mplot = ModelPlot()

dplot.prepare(fi.data)
dplot.histo_prefs['linestyle'] = '-'
mplot.prepare(fi.data, fi.model)
mplot.plot_prefs['color'] = 'red'
fplot.prepare(dplot, mplot)
splot.addplot(fplot,ylog=True)
rplot.prepare(fi.data, fi.model, fi.stat)
splot.addplot(rplot)
# the need for a hline call is due to bug #378
rplot.hline(0, linecolor='gray', linestyle='dot', overplot=True)

## Error Analysis

In [ ]:
coverrs = fi.est_errors()
# print(coverrs)
print(coverrs.format())

coverrs.extra_output # covariance matrix

temp_errs = fi.est_errors(parlist=(mdl_rsp.model.pars[0], ))
em_errs = fi.est_errors(parlist=(mdl_rsp.model.pars[1], ))


In [ ]:
iproj = IntervalProjection()
iproj.prepare(fac=5, nloop=51)
iproj.calc(fi, mdl_rsp.model.pars[0])
iproj.plot()

pmin = temp_errs.parvals[0] + temp_errs.parmins[0]
pmax = temp_errs.parvals[0] + temp_errs.parmaxes[0]
iproj.vline(pmin, overplot=True, linestyle='dot')
iproj.vline(pmax, overplot=True, linestyle='dot')

In [ ]:
iproj = IntervalProjection()
iproj.prepare(fac=5, nloop=51)
iproj.calc(fi, mdl_rsp.model.pars[1])
iproj.plot()

pmin = em_errs.parvals[0] + em_errs.parmins[0]
pmax = em_errs.parvals[0] + em_errs.parmaxes[0]
iproj.vline(pmin, overplot=True, linestyle='dot')
iproj.vline(pmax, overplot=True, linestyle='dot')

In [ ]:
rproj = RegionProjection()
rproj.calc(fi, mdl_rsp.model.pars[0], mdl_rsp.model.pars[1])
rproj.contour()

## Pre-Flare

In [ ]:
alltime, allflux_1_3, allcount = xsm_gen_lc(pha_file,1,3)
alltime, allflux_3_6, allcount = xsm_gen_lc(pha_file,3,6)
alltime, allflux_6_10, allcount = xsm_gen_lc(pha_file,6,10)
alltime, allflux_10_15, allcount = xsm_gen_lc(pha_file,10,15)
# alltime = alltime - alltime.min()
fig, ax = plt.subplots()
ax.semilogy(alltime[600:900],allflux_1_3[600:900],label=r'$1 - 3$ keV')
ax.semilogy(alltime[600:900],allflux_3_6[600:900],label=r'$3 - 6$ keV')
ax.semilogy(alltime[600:900],allflux_6_10[600:900],label=r'$6 - 10$ keV')
ax.semilogy(alltime[600:900],allflux_10_15[600:900],label=r'$10 - 15$ keV')
ax.axvspan(alltime[640],alltime[655],alpha=0.5, color='black')
ax.set_xlabel('Time (s)')
ax.set_ylabel(r'Flux (W/m$^2$)')
ax.legend()
ax.xaxis.set_tick_params(labelsize=12)
ax.set_title('M4.0 Flare (UT)');

In [ ]:
xsm_genspec(pha_file,alltime[640],alltime[655],'ch2_xsm_20220328_v1_level2_preflare')

In [ ]:
pha_pre = read_pha('ch2_xsm_20220328_v1_level2_preflare.pha',use_errors=True)
print(pha_pre)

In [ ]:
pha_pre = read_pha('ch2_xsm_20220328_v1_level2_preflare.pha',use_errors=True)
pha_pre.set_rmf(rmf)
pha_pre.set_analysis('energy')
pha_pre.notice(1.5,7)

xsm_rsp_pre = Response1D(pha_pre)
mdl_rsp_pre = xsm_rsp_pre(mdl)
print(mdl_rsp_pre)

pha_pre.staterror[pha_pre.staterror==0] = 1
pha_pre.syserror[pha_pre.syserror == 0] = 1

mdl_rsp_pre.model.pars[0].val = 10 # Temperature
mdl_rsp_pre.model.pars[1].val = 0.01 #EM

fi_pre = Fit(pha_pre,mdl_rsp_pre)

fres_pre = fi_pre.fit()
print(fres_pre)
print(f'Reduced Chi2 = {fres_pre.statval/fres_pre.dof:.4f}')

In [ ]:
fplot = FitPlot()
rplot = DelchiPlot()
splot = SplitPlot()
dplot = DataPlot()
mplot = ModelPlot()

dplot.prepare(fi_pre.data)
mplot.prepare(fi_pre.data, fi_pre.model)
mplot.plot_prefs['color'] = 'red'
fplot.prepare(dplot, mplot)
splot.addplot(fplot,ylog=True)
rplot.prepare(fi_pre.data, fi_pre.model, fi_pre.stat)
splot.addplot(rplot)
# the need for a hline call is due to bug #378
rplot.hline(0, linecolor='gray', linestyle='dot', overplot=True)

## Non-Thermal

In [ ]:
class f_thick2_sherpa(model.ArithmeticModel):
    def __init__(self,name='f_thick2'):
        self.em = model.Parameter(name,'EM',1,min=1e-5,hard_min=1e-49)
        self.p = model.Parameter(name,'p',4,min=1,hard_min=0)
        self.eebrk = model.Parameter(name,'eebrk',150,min=1,hard_min=0)
        self.q = model.Parameter(name,'q',6,min=1,hard_min=0)
        self.eelow = model.Parameter(name,'eelow',20,min=1,hard_min=0)
        self.eehigh = model.Parameter(name,'eehigh',32000,min=1,hard_min=0)
        model.ArithmeticModel.__init__(self, name,(self.em,self.p,self.eebrk,self.q,self.eelow,self.eehigh))

    def calc(self,pars,x,*args,**kwargs):

        ene = x
        em, p, eebrk, q, eelow, eehigh = pars

        if not (eelow <= eebrk <= eehigh):
            return np.zeros(len(x))

        em = em*1e35

        y = emission.bremsstrahlung_thick_target(ene, p, eebrk, q, eelow, eehigh)*em

        return y

mdl_thick2 = f_thick2_sherpa() 
mdl_expr_thick2 = xsm_rsp(mdl_thick2)
print(mdl_expr_thick2)

mdl2 = f_vth_sherpa()
mdl_vth_rsp = xsm_rsp(mdl2)
print(mdl_vth_rsp)

In [ ]:
fig, ax = plt.subplots()
ene_xx = np.arange(1,100)
ax.semilogy(ene_xx,mdl_thick2.calc([1,4,150,6,30,32000],ene_xx),label='Index = 4, Break Ene = 30keV')
ax.semilogy(ene_xx,mdl_thick2.calc([1,7,150,6,30,32000],ene_xx),label='Index = 7, Break Ene = 30keV')
ax.semilogy(ene_xx,mdl_thick2.calc([1,4,150,6,20,32000],ene_xx),label='Index = 4, Break Ene = 20keV')
ax.semilogy(ene_xx,mdl_thick2.calc([1,6,150,6,20,32000],ene_xx),label='Index = 6, Break Ene = 20keV')
ax.semilogy(ene_xx,mdl_thick2.calc([1,5,150,6,10,32000],ene_xx),label='Index = 5, Break Ene = 10keV')
# ax.semilogy(ene_x,mdl.calc([10,1],ene_x),label='10 MK')
# ax.semilogy(ene_x,mdl.calc([20,1],ene_x),label='20 MK')
# ax.semilogy(ene_x,mdl.calc([30,1],ene_x),label='30 MK')
ax.legend()
ax.set_xlabel('Energy (keV)')
ax.set_ylabel('Flux')
# ax.set_xlim([1,20])
ax.set_xscale('log')

### Fitting

In [ ]:
alltime, allflux_1_3, allcount = xsm_gen_lc(pha_file,1,3)
alltime, allflux_3_6, allcount = xsm_gen_lc(pha_file,3,6)
alltime, allflux_6_10, allcount = xsm_gen_lc(pha_file,6,10)
alltime, allflux_10_15, allcount = xsm_gen_lc(pha_file,10,15)
# alltime = alltime - alltime.min()
fig, ax = plt.subplots()
ax.semilogy(alltime[600:900],allflux_1_3[600:900],label=r'$1 - 3$ keV')
ax.semilogy(alltime[600:900],allflux_3_6[600:900],label=r'$3 - 6$ keV')
ax.semilogy(alltime[600:900],allflux_6_10[600:900],label=r'$6 - 10$ keV')
ax.semilogy(alltime[600:900],allflux_10_15[600:900],label=r'$10 - 15$ keV')
ax.axvspan(alltime[680],alltime[683],alpha=0.5, color='black')
ax.set_xlabel('Time (s)')
ax.set_ylabel(r'Flux (W/m$^2$)')
ax.legend()
ax.xaxis.set_tick_params(labelsize=12)
ax.set_title('M4.0 Flare (UT)');

In [ ]:
xsm_genspec(pha_file,alltime[680],alltime[683],'ch2_xsm_20220328_v1_level2_nonth')

#### Thermal

In [ ]:
pha = read_pha('ch2_xsm_20220328_v1_level2_nonth.pha',use_errors=True)
pha.set_rmf(rmf)
pha.set_analysis('energy')
pha.notice(1.5,15)

pha.staterror[pha.staterror==0] = 1
pha.syserror[pha.syserror == 0] = 1

mdl_tmp = f_vth_sherpa()
mdl_rsp = xsm_rsp(mdl_tmp)

mdl_rsp.model.pars[0].val = 10 # Temperature
mdl_rsp.model.pars[1].val = 0.01 #EM

fi_th = Fit(pha,mdl_rsp)

fres_th = fi_th.fit()
print(fres_th)
print(f'Reduced Chi2 = {fres_th.statval/fres_th.dof:.4f}')

In [ ]:
fplot = FitPlot()
rplot = DelchiPlot()
splot = SplitPlot()
dplot = DataPHAPlot()
mplot = ModelPlot()

dplot.prepare(fi_th.data)
dplot.histo_prefs['linestyle'] = '-'
mplot.prepare(fi_th.data, fi_th.model)
mplot.plot_prefs['color'] = 'red'
fplot.prepare(dplot, mplot)
splot.addplot(fplot,ylog=True)
rplot.prepare(fi_th.data, fi_th.model, fi_th.stat)
splot.addplot(rplot)
# the need for a hline call is due to bug #378
rplot.hline(0, linecolor='gray', linestyle='dot', overplot=True)

#### Fitting with Thermal + Non-Thermal

In [ ]:
mdl_thick2 = f_thick2_sherpa() 
mdl_expr_thick2 = xsm_rsp(mdl_thick2)
print(mdl_expr_thick2)

mdl2 = f_vth_sherpa()
mdl_vth_rsp = xsm_rsp(mdl2)
print(mdl_vth_rsp)

mdl_expr_thick2.model.pars[5].frozen = True
mdl_expr_thick2.model.pars[2].frozen = True
mdl_expr_thick2.model.pars[3].frozen = True
mdl_expr_thick2.model.pars[1].frozen = False
mdl_expr_thick2.model.pars[4].frozen = False
mdl_expr_thick2.model.pars[0].frozen = False

mdl_expr_thick2.model.pars[4].val = 20
mdl_expr_thick2.model.pars[0].val = 10
mdl_expr_thick2.model.pars[1].val = 4

mdl_vth_rsp.model.pars[0].frozen = False
mdl_vth_rsp.model.pars[1].frozen = False

In [ ]:
pha = read_pha('ch2_xsm_20220328_v1_level2_nonth.pha',use_errors=True)
pha.set_rmf(rmf)
pha.set_analysis('energy')
pha.notice(1.5,15)

pha.staterror[pha.staterror==0] = 1
pha.syserror[pha.syserror == 0] = 1

# mdl_rsp.model.pars[0].val = 10 # Temperature
# mdl_rsp.model.pars[1].val = 0.01 #EM

fi_t2 = Fit(pha,mdl_expr_thick2+mdl_vth_rsp)

fres_t2 = fi_t2.fit()
print(fres_t2)
print(f'Reduced Chi2 = {fres_t2.statval/fres_t2.dof:.4f}')

In [ ]:
fplot = FitPlot()
rplot = DelchiPlot()
splot = SplitPlot()
dplot = DataPlot()
mplot = ModelPlot()

dplot.prepare(fi_t2.data)
mplot.prepare(fi_t2.data, fi_t2.model)
mplot.plot_prefs['color'] = 'red'
fplot.prepare(dplot, mplot)
splot.addplot(fplot,ylog=True)
rplot.prepare(fi_t2.data, fi_t2.model, fi_t2.stat)
splot.addplot(rplot)
# the need for a hline call is due to bug #378
rplot.hline(0, linecolor='gray', linestyle='dot', overplot=True)

#### Comparison

In [ ]:
dplot = DataPlot()
dplot.prepare(fi_th.data)
mplot1 = ModelPlot()
mplot2 = ModelPlot()
mplot1.prepare(fi_th.data, fi_th.model)
mplot2.prepare(fi_t2.data, fi_t2.model)
dplot.plot(ylog=True)
mplot1.overplot(color='red')

mplot2.prepare(fi_t2.data, fi_t2.model)
mplot2.overplot()#(color='red')
# mplot2.plot_prefs['color'] = 'red'

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(ene_x,pha.y)
ax.semilogy(ene_x,mdl_vth_rsp.calc(fres_t2.parvals[-2:],ene_x),label='Thermal')
ax.semilogy(ene_x,mdl_expr_thick2.calc([fres_t2.parvals[0],fres_t2.parvals[1],150,6,fres_t2.parvals[2],32000],ene_x),label='Non_thermal')
ax.semilogy(ene_x,mdl_vth_rsp.calc(fres_t2.parvals[-2:],ene_x)+mdl_expr_thick2.calc([fres_t2.parvals[0],fres_t2.parvals[1],150,6,fres_t2.parvals[2],32000],ene_x),label='Total')
ax.legend()

# Temporal Evolution

In [ ]:
alltime, allflux_1_3, allcount = xsm_gen_lc(pha_file,1,3)
alltime, allflux_3_6, allcount = xsm_gen_lc(pha_file,3,6)
alltime, allflux_6_10, allcount = xsm_gen_lc(pha_file,6,10)
alltime, allflux_10_15, allcount = xsm_gen_lc(pha_file,10,15)
# alltime = alltime - alltime.min()
fig, ax = plt.subplots()
ax.semilogy(alltime[600:800],allflux_1_3[600:800],label=r'$1 - 3$ keV')
ax.semilogy(alltime[600:800],allflux_3_6[600:800],label=r'$3 - 6$ keV')
ax.semilogy(alltime[600:800],allflux_6_10[600:800],label=r'$6 - 10$ keV')
ax.semilogy(alltime[600:800],allflux_10_15[600:800],label=r'$10 - 15$ keV')
ax.axvspan(alltime[665],alltime[750],alpha=0.5, color='black')
ax.set_xlabel('Time (s)')
ax.set_ylabel(r'Flux (W/m$^2$)')
# ax.legend()
ax.xaxis.set_tick_params(labelsize=12)
ax.set_title('M4.0 Flare (UT)');

In [ ]:
!mkdir tmp_spectra

In [ ]:
j = 0
spec_time = []
for ii in range(640,750,5):
    print(f'tmp_spectra/ch2_xsm_20220328_v1_level2_flare_{j}')
    xsm_genspec(pha_file,alltime[ii],alltime[ii+5],f'tmp_spectra/ch2_xsm_20220328_v1_level2_flare_{j}')
    j = j+1
    spec_time.append(np.mean([alltime[ii],alltime[ii+5]]))

In [ ]:
fres_all = {}

for j in range(22):
    print(f'tmp_spectra/ch2_xsm_20220328_v1_level2_flare_{j}.pha')
    pha = read_pha(f'tmp_spectra/ch2_xsm_20220328_v1_level2_flare_{j}.pha',use_errors=True)
    pha.set_rmf(rmf)
    pha.set_analysis('energy')
    pha.notice(1.5,15)

    mdl_tmp = f_vth_sherpa()
    xsm_rsp = Response1D(pha)
    mdl_rsp_tmp = xsm_rsp(mdl_tmp)

    pha.staterror[pha.staterror==0] = 1
    pha.syserror[pha.syserror == 0] = 1

    mdl_rsp_tmp.model.pars[0].val = 10 # Temperature
    mdl_rsp_tmp.model.pars[1].val = 0.01 #EM

    fi_tmp = Fit(pha,mdl_rsp_tmp)

    fres_tmp = fi_tmp.fit()
    fres_all[j] = fres_tmp
#     print(fres_tmp)
#     print(f'Reduced Chi2 = {fres_tmp.statval/fres_tmp.dof:.4f}')

In [ ]:
temps = []
ems = []

for j in range(22):
    temps.append(fres_all[j].parvals[0])
    ems.append(fres_all[j].parvals[1])

In [ ]:
fig, axes = plt.subplots(2,1,sharex=True)

ax = axes[0]
ax.step(spec_time,temps,label='Temperature')
# ax.set_ylabel()

ax = axes[1]
ax.step(spec_time,ems,label='EM')
ax.xaxis.set_tick_params(labelsize=12)
